In [5]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit import execute, Aer
# from qc_grader import prepare_ex3, grade_ex3
import numpy as np
from heapq import nlargest


In [6]:

problem_set = [
    [["0", "2"], ["1", "0"], ["1", "2"], ["1", "3"], ["2", "0"], ["3", "3"]],
    [["0", "0"], ["0", "1"], ["1", "2"], ["2", "2"], ["3", "0"], ["3", "3"]],
    [["0", "0"], ["1", "1"], ["1", "3"], ["2", "0"], ["3", "2"], ["3", "3"]],
    [["0", "0"], ["0", "1"], ["1", "1"], ["1", "3"], ["3", "2"], ["3", "3"]],
    [["0", "2"], ["1", "0"], ["1", "3"], ["2", "0"], ["3", "2"], ["3", "3"]],
    [["1", "1"], ["1", "2"], ["2", "0"], ["2", "1"], ["3", "1"], ["3", "3"]],
    [["0", "2"], ["0", "3"], ["1", "2"], ["2", "0"], ["2", "1"], ["3", "3"]],
    [["0", "0"], ["0", "3"], ["1", "2"], ["2", "2"], ["2", "3"], ["3", "0"]],
    [["0", "3"], ["1", "1"], ["1", "2"], ["2", "0"], ["2", "1"], ["3", "3"]],
    [["0", "0"], ["0", "1"], ["1", "3"], ["2", "1"], ["2", "3"], ["3", "0"]],
    [["0", "1"], ["0", "3"], ["1", "2"], ["1", "3"], ["2", "0"], ["3", "2"]],
    [["0", "0"], ["1", "3"], ["2", "0"], ["2", "1"], ["2", "3"], ["3", "1"]],
    [["0", "1"], ["0", "2"], ["1", "0"], ["1", "2"], ["2", "2"], ["2", "3"]],
    [["0", "3"], ["1", "0"], ["1", "3"], ["2", "1"], ["2", "2"], ["3", "0"]],
    [["0", "2"], ["0", "3"], ["1", "2"], ["2", "3"], ["3", "0"], ["3", "1"]],
    [["0", "1"], ["1", "0"], ["1", "2"], ["2", "2"], ["3", "0"], ["3", "1"]],
]

In [7]:

def diffusion(qc: QuantumCircuit, qubits: QuantumRegister, qr_extra: QuantumRegister):

    qc.h(qubits)
    qc.x(qubits)
    qc.h(qubits[-1])
    try:
        qc.mct(qubits[:-1], qubits[-1], qr_extra, mode="v-chain")
    except ValueError:
        qc.mct(qubits[:-1], qubits[-1], qr_extra, mode="recursion")
    qc.h(qubits[-1])
    qc.x(qubits)
    qc.h(qubits)

    qc.barrier()

    return qc


def mark_address(qc: QuantumCircuit, qubits: QuantumRegister, index: int):

    if index not in range(2 ** len(qubits)):
        raise ValueError(f"index is {index} must be in range(16)")

    # if len(qubits) != 4:
    #    raise ValueError("qubits must have length of 4")

    # Convert index to a binary string
    if len(qubits) == 1:
        bin_str = f"{index:01b}"
    if len(qubits) == 2:
        bin_str = f"{index:02b}"
    if len(qubits) == 4:
        bin_str = f"{index:04b}"

    for i in range(len(qubits)):
        if bin_str[i] == "0":
            qc.x(qubits[i])

    return qc


def game_logic_oracle(
    qr_shots: QuantumRegister,
    qr_address: QuantumRegister,
    qr_cluster: QuantumRegister,
    qr_extra: QuantumRegister,
    prob_set,
    return_gate: bool = True,
):

    qc = QuantumCircuit(qr_shots, qr_address, qr_cluster, qr_extra)

    for p, puzzle in enumerate(prob_set):  # for each puzzle

        puzzle = np.array(puzzle, int)  # convert to int
        # Mark the puzzle's address (p):
        qc = mark_address(qc, qr_address, p)

        for c, (h, v) in enumerate(puzzle):  # for each position pair in puzzle
            # Control cluster c with address and the puzzle's horizontal number
            qc.mct(
                qr_address[:] + [qr_shots[h]], qr_cluster[c], qr_extra, mode="v-chain",
            )
            qc.mct(
                qr_address[:] + [qr_shots[v + 4]],
                qr_cluster[c],
                qr_extra,
                mode="v-chain",
            )

        # Unmark the puzzles's address:
        qc = mark_address(qc, qr_address, p)

    # Convert to gate and return
    if return_gate:

        game_logic = qc.to_gate()
        game_logic.name = "Game Logic"
        return game_logic
    else:
        return qc




In [8]:
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, execute, Aer
import numpy as np


def qft_dagger(n):
    """n-qubit QFTdagger the first n qubits in circ"""
    circ = QuantumCircuit(n)
    # Don't forget the Swaps!
    for qubit in range(n // 2):
        circ.swap(qubit, n - qubit - 1)
    for j in range(n):
        for m in range(j):
            circ.cp(-np.pi / float(2 ** (j - m)), m, j)
        circ.h(j)

    qft_dag = circ.to_gate()
    qft_dag.name = "$QFT_dag$"
    return qft_dag


def count_oracle(
    qr_count_me: QuantumRegister, qr_counter: QuantumRegister, return_gate: bool = True,
):
    """ Creates a gate/circuit that counts the number of 1's in
    qr_count_me and writes to qr_counter

    ----------
    PARAMETERS
    ----------
    qr_count_me:
        A quantum register whose 1's will be counted
    qr_counter:
        A quantum register that will record the count
    return_gate:
        Returns gate if true, quantum circuit if false.

    -------
    RETURNS
    -------
    A gate or quantum circuit that counts qr_count_me and writes to qr_counter

    """

    qc = QuantumCircuit(qr_count_me, qr_counter)

    # Step 1: Put the counter into superposition
    qc.h(qr_counter)

    for qubit_count_me in qr_count_me:
        for i, qubit_counter in enumerate(qr_counter):
            # For each less significant qubit, we need to do a
            # smaller-angled controlled rotation:
            qc.cp(np.pi / 2 ** (len(qr_counter) - i - 1), qubit_count_me, qubit_counter)

    qc.append(qft_dagger(len(qr_counter)), qargs=qr_counter)

    if return_gate:
        counter_gate = qc.to_gate()
        counter_gate.name = "Counter"
        return counter_gate
    else:
        return qc


if __name__ == "__main__":

    count_me_size = 8
    counter_size = 4

    qr_count_me = QuantumRegister(count_me_size)
    qr_counter = QuantumRegister(counter_size)

    cr_count_me = ClassicalRegister(count_me_size)
    cr_counter = ClassicalRegister(counter_size)

    qc = QuantumCircuit(qr_count_me, qr_counter, cr_count_me, cr_counter)

    # Put qr_count_me into superposition
    qc.h(qr_count_me)

    counter_gate = count_oracle(
        qr_count_me=qr_count_me, qr_counter=qr_counter, return_gate=True
    )

    qc.append(
        counter_gate, qr_count_me[:] + qr_counter[:],
    )

    qc.measure(qr_count_me, cr_count_me)
    qc.measure(qr_counter, cr_counter)

    backend = Aer.get_backend("qasm_simulator")
    job = execute(qc, backend=backend, shots=2000,)

    result = job.result()
    count = result.get_counts()

    for k, v in count.items():
        print(k, v)


0000 00000000 9
0001 00000001 4
0001 00000010 5
0001 00000100 4
0001 00001000 7
0001 00010000 11
0001 00100000 7
0001 01000000 8
0001 10000000 8
0010 00000011 9
0010 00000101 5
0010 00000110 9
0010 00001001 8
0010 00001010 12
0010 00001100 16
0010 00010001 7
0010 00010010 2
0010 00010100 8
0010 00011000 7
0010 00100001 3
0010 00100010 10
0010 00100100 7
0010 00101000 7
0010 00110000 2
0010 01000001 8
0010 01000010 10
0010 01000100 6
0010 01001000 8
0010 01010000 9
0010 01100000 5
0010 10000001 13
0010 10000010 5
0010 10000100 6
0010 10001000 10
0010 10010000 4
0010 10100000 7
0010 11000000 9
0011 00000111 9
0011 00001011 10
0011 00001101 12
0011 00001110 6
0011 00010011 4
0011 00010101 9
0011 00010110 7
0011 00011001 7
0011 00011010 9
0011 00011100 9
0011 00100011 8
0011 00100101 1
0011 00100110 12
0011 00101001 4
0011 00101010 7
0011 00101100 10
0011 00110001 11
0011 00110010 8
0011 00110100 6
0011 00111000 11
0011 01000011 7
0011 01000101 9
0011 01000110 9
0011 01001001 9
0011 010010

In [43]:

def week3_ans_func(problem_set, count_shots=False):

    num_problems = len(problem_set)
    address_qubits = int(np.sqrt(num_problems))
    counting_qubits = 4

    # 4 + 4 (shot options) + 6 (clusters) + 4 (address) + 3 (counting) + 1 (ancilla) = 22, 6 extra!
    qr_shots = QuantumRegister(8, name="shots")
    qr_cluster = QuantumRegister(6, name="clusters")
    qr_address = QuantumRegister(address_qubits, name="address")
    qr_counter = QuantumRegister(counting_qubits, name="counting")
    qr_ancilla = QuantumRegister(2, name="ancilla")
    num_extra = 28 - len(
        qr_shots[:] + qr_cluster[:] + qr_address[:] + qr_counter[:] + qr_ancilla[:]
    )

    qr_extra = QuantumRegister(num_extra, name="extra")

    cr_shots = ClassicalRegister(len(qr_shots))
    cr_address = ClassicalRegister(address_qubits)

    if count_shots:
        qc = QuantumCircuit(
            qr_shots,
            qr_address,
            qr_cluster,
            qr_counter,
            qr_ancilla,
            qr_extra,
            cr_shots,
        )
    else:
        qc = QuantumCircuit(
            qr_shots,
            qr_address,
            qr_cluster,
            qr_counter,
            qr_ancilla,
            qr_extra,
            cr_address,
        )

    # Set cluster status to 0
    # No code required

    # Prepare ancilla
    qc.x(qr_ancilla)
    qc.h(qr_ancilla)

    # Put solution into superposition
    qc.h(qr_shots[:] + qr_address[:])

    qc.barrier()

    # Load game gate
    game_qubits = qr_shots[:] + qr_address[:] + qr_cluster[:] + qr_extra[:]

    game_gate = game_logic_oracle(
        qr_shots=qr_shots,
        qr_address=qr_address,
        qr_cluster=qr_cluster,
        qr_extra=qr_extra,
        prob_set=problem_set,
    )

    # Load counter gate
    counter_qubits = qr_shots[:] + qr_counter[:]
    counter_gate = count_oracle(
        qr_count_me=qr_shots, qr_counter=qr_counter, return_gate=True
    )

    game_iterations = 1
    # Code for Grover's algorithm with iterations = 1 will be as follows.
    for _ in range(1):

        # Perform Grover's algorithm for game logic
        for j in range(game_iterations):

            # Append game gate
            qc.append(game_gate, game_qubits)

            # Check solution
            try:
                qc.mct(qr_cluster, qr_ancilla[0], qr_extra, mode="v-chain")
            except ValueError:
                qc.mct(qr_cluster, qr_ancilla[0], qr_extra, mode="recursion")

            # Uncompute game gate
            qc.append(game_gate.inverse(), game_qubits)

            # Diffusion circuit
            qc = diffusion(qc, qr_shots, qr_extra)

        # Append counter gate
        qc.append(counter_gate, counter_qubits)

        # Mark the desired state: Exactly 4: 0100
        qc.x(qr_counter[0:2])
        if counting_qubits >= 4:
            qc.x(qr_counter[3])

        # Check for the solution
        #qc.h(qr_ancilla[1])
        qc.mct(qr_counter, qr_ancilla[0], qr_extra, mode="recursion") # , mode="v-chain")
        #qc.h(qr_ancilla[1])

        # Unmark the desired state
        qc.x(qr_counter[0:2])
        if counting_qubits >= 4:
            qc.x(qr_counter[3])

        # Uncompute counter gate
        qc.append(counter_gate.inverse(), counter_qubits)

        # Reverse Grover's algorithm for game logic
        for j in range(game_iterations):

            # Diffusion circuit
            qc = diffusion(qc, qr_shots, qr_extra)

            # Append game gate
            qc.append(game_gate.inverse(), game_qubits)

            # Check solution
            try:
                qc.mct(qr_cluster, qr_ancilla[0], qr_extra, mode="v-chain")
            except ValueError:
                qc.mct(qr_cluster, qr_ancilla[0], qr_extra, mode="recursion")

            # Uncompute game gate
            qc.append(game_gate, game_qubits)

        # Diffusion circuit
        qc = diffusion(qc, qr_address, qr_extra)

    # Measure results
    if count_shots:
        qc.measure(qr_shots, cr_shots[::-1])
    else:
        qc.measure(qr_address, cr_address)

    return qc


In [45]:
from test_data import q1, q2, q3

qc = week3_ans_func(q3[0][0:4], count_shots=False)

backend = Aer.get_backend("qasm_simulator")
tic = time.perf_counter()
job = execute(
    qc,
    backend=backend,
    shots=20000,
    optimization_level=3,
    backend_options={"fusion_enable": True},
)

result = job.result()
count = result.get_counts()

for k, v in count.items():
    print(k, v)

toc = time.perf_counter()

print(f"Circuit executed in {toc - tic:0.4f} seconds")

00 4652
01 4808
10 4753
11 5787
Circuit executed in 124.3524 seconds
